In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# A simple case.

Consider a simple case with 3 astronauts. Each astronaut can process folate at baseline, above baseline, and below baseline levels.
Consider two possible foods to send to ISS, an apple and a banana. Each has an amount of folate per unit.

Our goal is to compute the *optimum* amount of each food to bring for the entire mission: x.

In [2]:
astro_simple = [1.0, 1.5, 0.5]

APPLE  = 0;
BANANA = 1;

foods_simple = [0.001, .05]

x = [x for x in range(0, len(astro_simple) * len(foods_simple))]

In [3]:
# We consruct a pyomo optimization model to find the optimal basket of goods to take to space that minimizes the total amount of units
# taken.

model = pyo.ConcreteModel()
model.x = pyo.Var(x, domain=pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr = sum(model.x))

There's constraints however; each astronaut needs at least 5 units of folate to survive.

In [4]:
# An astronauts total folate processed is their innate metabolism * (# of apples * apple folate + # of bananas * banana folate).
model.Constraint1 = pyo.Constraint(expr = astro_simple[0] * (model.x[0] * foods_simple[0] + model.x[1] * foods_simple[1]) >= 5)
model.Constraint2 = pyo.Constraint(expr = astro_simple[1] * (model.x[2] * foods_simple[0] + model.x[3] * foods_simple[1]) >= 5)
model.Constraint3 = pyo.Constraint(expr = astro_simple[2] * (model.x[4] * foods_simple[0] + model.x[5] * foods_simple[1]) >= 5)

We can then find a solution that produces the minimum amount of foods we need to take to satisfy our constraint.

In [5]:
opt = SolverFactory("glpk")
result = opt.solve(model)

print("ASTRO_1 APPLES: {}, ASTRO_1 BANANAS: {}, ASTRO_2 APPLES: {}, ASTRO_2 BANANAS: {}, ASTRO_3 APPLES: {}, ASTRO_3 BANANAS: {}"
      .format(model.x[0].value, model.x[1].value, model.x[2].value, model.x[3].value, model.x[4].value, model.x[5].value))

ASTRO_1 APPLES: 5000.0, ASTRO_1 BANANAS: 0.0, ASTRO_2 APPLES: 3333.33333333333, ASTRO_2 BANANAS: 0.0, ASTRO_3 APPLES: 10000.0, ASTRO_3 BANANAS: 0.0


This isn't too helpful, it just says we should send all apples; we can add more constraints to make this more lifelike.

In [6]:
# We can only send 10000 apples in total.
model.Constraint4 = pyo.Constraint(expr = model.x[0] + model.x[2] <= 10000)
# And Astronaut 2 is only willing to eat 100 bananas!
model.Constraint5 = pyo.Constraint(expr = model.x[3] <= 100)

In [7]:
# Solve again.

opt = SolverFactory("glpk")
result = opt.solve(model)

print("ASTRO_1 APPLES: {}, ASTRO_1 BANANAS: {}, ASTRO_2 APPLES: {}, ASTRO_2 BANANAS: {}, ASTRO_3 APPLES: {}, ASTRO_3 BANANAS: {}"
      .format(model.x[0].value, model.x[1].value, model.x[2].value, model.x[3].value, model.x[4].value, model.x[5].value))

ASTRO_1 APPLES: 0.0, ASTRO_1 BANANAS: 100.0, ASTRO_2 APPLES: 0.0, ASTRO_2 BANANAS: 66.6666666666667, ASTRO_3 APPLES: 10000.0, ASTRO_3 BANANAS: 0.0


You can see how this changes the solution. This simple system are the beginning steps toward a much more powerful program.